# BERT를 이용한 한국어 텍스트 분류(허깅페이스 최소 활용)

## Install Hugging Face Transformer

## 선학습 모델 설정

- https://huggingface.co/models?library=pytorch&pipeline_tag=fill-mask&sort=downloads&search=kc

In [1]:
# 아래 선학습 파라미터는 beomi/kcbert-large로 학습한 것

# pretrained_mode_name = "beomi/kcbert-base"
pretrained_mode_name = "beomi/kcbert-large"

In [2]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification #, AdamW , BertConfig
from transformers import get_linear_schedule_with_warmup

import torch
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime
import re

from tqdm import tqdm

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Counting objects: 100% (14762/14762), done.
remote: Compressing objects: 100% (13012/13012), done.
remote: Total 14763 (delta 1748), reused 14762 (delta 1748), pack-reused 1 (from 1)
Receiving objects: 100% (14763/14763), 56.19 MiB | 21.48 MiB/s, done.
Resolving deltas: 100% (1748/1748), done.
Updating files: 100% (14737/14737), done.


In [5]:
# 훈련셋과 테스트셋 데이터 로드
# 편의상 판다스 사용
train_df = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
test_df = pd.read_csv("nsmc/ratings_test.txt", sep='\t')

print(train_df.shape)
print(test_df.shape)

(150000, 3)
(50000, 3)


In [6]:
# info()를 찍어보면 document에 5개 null이 있다는 것을 알 수 있음
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [7]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [8]:
train_df[train_df['document'].isnull()]

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [9]:
test_df[test_df['document'].isnull()]

,id,document,label
5746,402110,NaN,1
7899,5026896,NaN,0
27097,511097,NaN,1


In [10]:
# null 데이터 확인
# 원래 데이터에서는 ''인 빈문자열인데 pandas가 로딩하면서 nan으로 바꾼것
train_df.loc[25857,'document']

nan

In [11]:
# 그냥 dropna()
train_df = train_df.dropna()
test_df = test_df.dropna()

In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [13]:
# 리뷰 문장만 추출
sentences_sr = train_df['document']

In [14]:
# 리뷰 문장을 담은 시리즈
sentences_sr

,document
0,아 더빙.. 진짜 짜증나네요 목소리
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2,너무재밓었다그래서보는것을추천한다
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
...,...
149995,인간이 문제지.. 소는 뭔죄인가..
149996,평점이 너무 낮아서...
149997,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?
149998,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상


## 토큰화

- 허깅페이스에서 제공하는 BertTokenizer를 사용하여 토큰화 시도

- 형태소 분석같은 언어 종속적인 사전 지식을 사용하지 않음

- WordPiece 알고리즘 적용하여 분절 시도

In [15]:
# bert에서 제공하는 토크나이저 초기화
tokenizer = BertTokenizer.from_pretrained(pretrained_mode_name, do_lower_case=False)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/672 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [16]:
# 토큰화 실험
output = tokenizer.encode("Hello, y'all! How are you 😁 ?")

# 숫자로 인코딩된 결과
print(output)

[2, 41, 4226, 18993, 4404, 15, 90, 10, 66, 18993, 5, 41, 23468, 66, 17107, 23380, 3850, 32, 3]


In [17]:
# 인코딩된 숫자를 토큰화
print( tokenizer.convert_ids_to_tokens(output) )

['[CLS]', 'H', '##e', '##ll', '##o', ',', 'y', "'", 'a', '##ll', '!', 'H', '##ow', 'a', '##re', 'you', '😁', '?', '[SEP]']


In [18]:
# 토큰화 직접 해보기

# 숫자를 토큰으로 바꾸는 사전
idx_to_tkn = { v:k for k, v in tokenizer.vocab.items()}

# 토큰화된 index를 토큰으로 변환하는 함수
def idx2tokens(encoding):
    return [ tuple(map(idx_to_tkn.get, output)) for output in encoding ]

print( idx2tokens([output]) )

[('[CLS]', 'H', '##e', '##ll', '##o', ',', 'y', "'", 'a', '##ll', '!', 'H', '##ow', 'a', '##re', 'you', '😁', '?', '[SEP]')]


- BERT에 입력으로 시작과 끝에 [CLS], [SEP]가 들어가야 하므로 자동으로 해당 토큰을 추가하는 것을 확인할 수 있음

### NSMC에 시험적으로 토큰화 적용

In [19]:
# BERT의 입력 형식에 맞게 변환
sentences_for_bert = ["[CLS] " + sentence + " [SEP]" for sentence in sentences_sr]
sentences_for_bert[25850:25855]

['[CLS] 난 이런 영화 좋더라........따뜻했어ㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓ [SEP]',
 '[CLS] 곽부성.오경이 아깝다.. OO이 만들어도 이것보단 낫겠다.. 진목승 발로 만들었나 [SEP]',
 '[CLS] 강시많이 나오게 해주지. [SEP]',
 '[CLS] 그넘의 깡패에다 억지감동 스크린쿼터제 개나 줘 [SEP]',
 '[CLS] 1편이 훨씬낫다 2편은 재미없다 별로 [SEP]']

In [20]:
# tokenizer.tokenize()는 숫자로 인코딩하지 않고 바로 토큰화해서 보여줌
tokenizer.tokenize(sentences_for_bert[25854])

['[CLS]', '1', '##편이', '훨씬', '##낫다', '2', '##편은', '재미', '##없다', '별로', '[SEP]']

In [21]:
# 토크나이저로 토큰화
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences_for_bert]

In [22]:
print("토크나이저로 들어가는 토큰화 전 문장: ", sentences_for_bert[0])
print("토큰화 된 문장: ", tokenized_texts[0])
print(type(tokenized_texts))

토크나이저로 들어가는 토큰화 전 문장:  [CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]
토큰화 된 문장:  ['[CLS]', '아', '더', '##빙', '.', '.', '진짜', '짜증나네', '##요', '목소리', '[SEP]']
<class 'list'>


- 전체 데이터 셋에 대해서 일괄적으로 토큰화 적용
- 토큰화 과정에서 부수적인 padding, truncation masking등을 함께 수행

In [23]:
# 문장 전체 길이 설정
MAX_LENGTH = 128

In [24]:
# 토크나이저를 바로 호출해서 토큰화, 패딩, 마스킹을 동시에 수행
# 예를 들면 이런식으로 토큰화 결과를 돌려줌(구체적인 숫자는 다를 수 있음)
# 입력 예: 토크나이저 실험중입니다.
# {
#   'input_ids': [101, 9873, 20308, 16439, 10739, 48387, 9489, 86834, 41693, 58303, 48345, 119, 102],
#   'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 첫번째 문장0, 두번째 문장 1 여기서는 두번째 문자없음
#   'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
# }

tkn_ecd_sentences = tokenizer(
    list(sentences_sr), # 시리즈를 리스트로 변환
    padding=True, truncation=True, # 패딩과 자르기
    max_length=MAX_LENGTH, # 최대 길이 지정
    return_tensors="pt",  # 반환 형식은 파이토치 텐서로
)

In [26]:
print(tkn_ecd_sentences.keys())

# 토크나이저가 반환하는 객체
type(tkn_ecd_sentences)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


transformers.tokenization_utils_base.BatchEncoding

In [27]:
tkn_ecd_sentences['input_ids'][25857:25857+2]

tensor([[    2,  7992,  4089,  5110,  8186,     5,     5,     5,  1052,  4064,
          4694,  9151,  9014,     5,  1052, 18715,     3,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [28]:
# convert_ids_to_tokens쓸려면 입력을 하니씩 넣어야 함
# https://github.com/huggingface/transformers/blob/v4.36.1/src/transformers/tokenization_utils.py#L957
ret = [tokenizer.convert_ids_to_tokens(x)
        for x in tkn_ecd_sentences['input_ids'][25857:25857+2].numpy()]

# 위에서 만들어둔 id2tokens를 쓰면 한번에 됨
# ret = idx2tokens(tkn_ecd_sentences['input_ids'][25857:25857+2].numpy())

print(ret)

[['[CLS]', '진짜', '##재', '##밋', '##어요', '!', '!', '!', '또', '##개', '##봉', '##했으면', '좋겠다', '!', '또', '##봐야지', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '

## train, valid set 분리

In [29]:
# train_df -> X_train, y_train, X_val, y_val
# test_df -> X_test, y_test

NUM_SAMPLES = 150000
# numpy로 바꿔서 NUM_SAMPLES만큼 잘라냄
train_np = train_df.to_numpy()
X_ = train_np[:NUM_SAMPLES, 1]
y_ = train_np[:NUM_SAMPLES, 2]
X_train, X_valid, y_train, y_valid = train_test_split(X_, y_,
                                        test_size=0.2, random_state=42)

print(f"X_train.shape: {X_train.shape}, type(X_train[0]): {type(X_train[0])}, \
y_train.shape: {y_train.shape}, type(y_train[0]): {type(y_train[0])}")

print(f"X_valid.shape: {X_valid.shape}, type(X_valid[0]): {type(X_valid[0])}, \
y_valid.shape: {y_valid.shape}, type(y_valid[0]): {type(y_valid[0])}")

X_train.shape: (119996,), type(X_train[0]): <class 'str'>, y_train.shape: (119996,), type(y_train[0]): <class 'int'>
X_valid.shape: (29999,), type(X_valid[0]): <class 'str'>, y_valid.shape: (29999,), type(y_valid[0]): <class 'int'>


In [30]:
# X_train은 numpy 어레이고 안에는 그냥 문장이 들어 있음
X_train[:2]

array(['아 꿀잼ㅋ 친구랑 봤는데 너무 웃겼음 그리구 김우빈 잘생겼다..',
       '개건의 졸작 스릴러? 스릴러라고 하게에도 민망하군'], dtype=object)

In [31]:
# test에도 똑같이 반복
test_np = test_df.to_numpy()
X_test = test_np[:, 1]
y_test = test_np[:, 2]

print(f"X_test.shape: {X_test.shape}, type(X_test[0]): {type(X_test[0])}, \
y_test.shape: {y_test.shape}, type(y_test[0]): {type(y_test[0])}")

X_test.shape: (49997,), type(X_test[0]): <class 'str'>, y_test.shape: (49997,), type(y_test[0]): <class 'int'>


## Dataset

In [32]:
class NsmcDataset(Dataset):

    def __init__(self, X, y):
        """
        X: 문장을 저장하고 있는 리스트, 문장은 토큰화 안된 상태
        y: 문장에 대한 레이블
        """
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        sample = str(self.X[i])
        label = self.y[i]

        return {
            'text': sample,
            'label': label,
        }


### collate_fn

In [33]:
# collate_fn
class MyCollate():
    def __init__(self, tokenizer, max_length):
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, minibatch):
        # minibatch는 Dataset에서 리턴한 {}의 리스트
        # return {
        #     'text': sample,
        #     'label': label,
        # }
        # 즉 minibatch는 Dataset이 리턴한 미니배치

        # 텍스트와 레이블을 따로 모으고
        texts = [ d['text'] for d in minibatch ]
        labels = [ d['label'] for d in minibatch ]

        # 외부에서 받은 토크나이저가 알아서 미니배치를 패딩함
        tkn_ecd_minibatch = self.tokenizer(
            texts,
            padding=True, truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        # 허깅 페이스 트랜스포머 모델의 전형적인 입력 형태로 변환해서 반환
        ret = {
            'input_ids': tkn_ecd_minibatch['input_ids'], # 토큰이 정수로 바뀐 리스트
            'attention_mask': tkn_ecd_minibatch['attention_mask'], # 패딩 위치를 저장하는 어텐션 마스크
            'labels': torch.tensor(labels, dtype=torch.long),
        }

        return ret


## DataLoader

In [34]:
# 시험삼아 데이터로더를 만들고 모델에 포워딩 테스트

m = 3

train_loader = DataLoader(
    NsmcDataset(X_train, y_train),
    batch_size=m,
    shuffle=True,
    # 데이터로더가 미니배치를 뽑을 때 콜레이트 함수가 작동하여
    # 토큰화, 패딩을 자동으로 처리
    collate_fn=MyCollate(tokenizer, MAX_LENGTH)
)

train_loader_iter = iter(train_loader)

In [35]:
# 미니배치중 제일 긴 샘플에 맞춰서 패딩되고 잘리는 것을 확인
# 정수화된 샘플들은 항상 2번[CLS]으로 시작해서 3번[SEP]로 끝남
data = next(train_loader_iter)
data

{'input_ids': tensor([[    2,  3288,  4213, 25932,    25, 28917, 28609,     3,     0,     0,
              0,     0,     0,     0,     0,     0],
         [    2, 10635,  4028,  7997, 13796,   786,  7988, 19898,   391,  4444,
           4091, 29257,  2173,  4601,  4126,     3],
         [    2,  8294,   809,  4240,  4042,  1293, 13069,  1279,  4019, 14946,
           5138,  4042,   254,  4240,     3,     0]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]]),
 'labels': tensor([0, 0, 1])}

## 모델

In [36]:
model = BertForSequenceClassification.from_pretrained(pretrained_mode_name,
                                                      num_labels=2)

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 포워드 테스트

In [37]:
# 포워드 테스트
output = model(**data)

# 입력을 구분해서 넣으려면 아래처럼 하면 됨
# model(data['input_ids'], attention_mask=data['attention_mask']).logits

In [38]:
# 로스와 로짓이 포함되어 있음
output

# 로짓은 (3,2)

SequenceClassifierOutput(loss=tensor(0.5000, grad_fn=<NllLossBackward0>), logits=tensor([[ 0.5192,  0.4295],
        [ 0.4250, -0.3057],
        [ 0.1609,  0.7054]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

## 선학습 파라미터 로딩

- 여기서 파라미터를 로딩했으면 학습은 하지말고 바로 테스트로 넘어가기

In [51]:
!gdown 1PpxZVkC7CO2DvM0ycWi_UYmuH69eEdX8

Downloading...
From (original): https://drive.google.com/uc?id=1PpxZVkC7CO2DvM0ycWi_UYmuH69eEdX8
From (redirected): https://drive.google.com/uc?id=1PpxZVkC7CO2DvM0ycWi_UYmuH69eEdX8&confirm=t&uuid=519a9657-6734-4e08-ac2c-f3b25b80d05e
To: /content/nsmc-kcbert-large.pth
100% 1.34G/1.34G [00:13<00:00, 102MB/s] 


In [52]:
model.load_state_dict(torch.load("nsmc-kcbert-large.pth", weights_only=True), strict=False)
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 1024, padding_idx=0)
      (position_embeddings): Embedding(300, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1

## 학습

In [39]:
m = 32

D_train = NsmcDataset(X_train, y_train)
train_loader = DataLoader(D_train, batch_size=m,
    shuffle=True, collate_fn=MyCollate(tokenizer, MAX_LENGTH))

D_valid = NsmcDataset(X_valid, y_valid)
valid_loader = DataLoader(D_valid, batch_size=m,
    collate_fn=MyCollate(tokenizer, MAX_LENGTH))

In [40]:
# optimizer and loss
# lr = 0.001
lr = 2e-5

epochs = 2

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, eps=1e-8)
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

In [41]:
# print info
print(f"mini-batch size: {m}")
print(f"epochs: {epochs}")
print(f"|D_train|: {len(D_train)}")
print(f"|D_valid|: {len(D_valid)}")


model.to(device)

for epoch in range(epochs):

    model.train()
    running_loss = 0.0
    corr = 0

    ##########################
    # TRAIN
    model.train()
    for i, data in enumerate(tqdm(train_loader)):
        X = data['input_ids'].to(device)
        mask = data['attention_mask'].to(device)
        y = data['labels'].to(device)

        optimizer.zero_grad()
        score = model(X, attention_mask=mask).logits
        loss = criterion(score, y)
        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss

        y_prob = torch.nn.functional.softmax(score, dim=-1)
        y_pred = torch.argmax(y_prob, dim=-1)
        corr += (y_pred == y).sum()
#         num_samples += y.shape[0]

    print("EPOCH:{:d}, Train Loss:{:.5f}, Acc.:{:.5f}".format(
            epoch+1,
            running_loss / len(train_loader),
            corr / len(D_train)))

    #######################
    # VALID
    model.eval()
    running_loss = 0.0
    corr = 0

    with torch.no_grad():
        for i, data in enumerate(tqdm(valid_loader)):
            X = data['input_ids'].to(device)
            mask = data['attention_mask'].to(device)
            y = data['labels'].to(device)

            score = model(X, attention_mask=mask).logits
            running_loss += loss

            y_prob = torch.nn.functional.softmax(score, dim=-1)
            y_pred = torch.argmax(y_prob, dim=-1)
            corr += (y_pred == y).sum()
#             num_samples += y.shape[0]

    print("EPOCH:{:d}, Valid Loss:{:.3f}, Acc.:{:.3f}".format(
            epoch+1,
            running_loss / len(valid_loader),
            corr / len(D_valid)))


mini-batch size: 32
epochs: 2
|D_train|: 119996
|D_valid|: 29999


100%|██████████| 3750/3750 [34:39<00:00,  1.80it/s]


EPOCH:1, Train Loss:0.26480, Acc.:0.88905


100%|██████████| 938/938 [02:49<00:00,  5.52it/s]


EPOCH:1, Valid Loss:0.176, Acc.:0.911


100%|██████████| 3750/3750 [34:36<00:00,  1.81it/s]


EPOCH:2, Train Loss:0.14055, Acc.:0.94737


100%|██████████| 938/938 [02:49<00:00,  5.52it/s]


EPOCH:2, Valid Loss:0.231, Acc.:0.915


## 테스트

In [42]:
D_test = NsmcDataset(X_test, y_test)
test_loader = DataLoader(D_test, batch_size=m,
    collate_fn=MyCollate(tokenizer, MAX_LENGTH))

In [43]:
corr = 0

with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        X = data['input_ids'].to(device)
        mask = data['attention_mask'].to(device)
        y = data['labels'].to(device)

        score = model(X, attention_mask=mask).logits
        y_prob = torch.nn.functional.softmax(score, dim=-1)
        y_pred = torch.argmax(y_prob, dim=-1)
        corr += (y_pred == y).sum()

test_acc = corr / len(D_test)
print(f"Test Acc.: {test_acc*100:.2f}%")

100%|██████████| 1563/1563 [04:35<00:00,  5.68it/s]

Test Acc.: 91.22%


- 다음 리뷰 문장에 대해서 테스트
    - "여러 단점이 있는데 배우들의 연기가 영화를 살렸습니다.",
    - "여러 단점이 있는데 배우들 연기도 망....",
    - "핵꿀잼",
    - "핵노잼"

In [53]:
test_sample = tokenizer(
            [
                "여러 단점이 있는데 배우들의 연기가 영화를 살렸습니다.",
                "여러 단점이 있는데 배우들 연기도 망....",
                "핵꿀잼",
                "핵노잼"
            ],
            padding=True, truncation=True,
            max_length=MAX_LENGTH,
            return_tensors="pt"
        )


score = model(test_sample['input_ids'].to(device),
              attention_mask=test_sample['attention_mask'].to(device)
        ).logits
y_prob = torch.nn.functional.softmax(score, dim=-1)
y_pred = torch.argmax(y_prob, dim=-1)
y_pred

tensor([1, 0, 1, 0], device='cuda:0')

## 모델 저장

In [ ]:
# torch.save(model.state_dict(), f"nsmc-kcbert-large.pth")